<div class="markdown-google-sans">

## **Installing required Package**
</div>

Because Salesforce-FuelSDK seems no longer maintained or more precisely, its suds dependency, we will install them independantly

In [ ]:
!pip install --no-deps Salesforce-FuelSDK
!pip install suds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.3 MB/s eta 0:00:00


Test Change

In [ ]:
from google.cloud import bigquery
import FuelSDK
import ET_Client
import pandas as pd
import pandas_gbq
import numpy as np
import datetime

Manually building client, if we want this to be used on a regular basis we should hide it using google secret

In [ ]:
today = datetime.date.today()
past_7_days = today - datetime.timedelta(days=7)

today_formatted = str(today.strftime('%Y-%m-%d'))+'T00:00:00.001'
past_7_days_formatted = str(past_7_days.strftime('%Y-%m-%d'))+'T00:00:00.001'

# SentEvent

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    stubObj.soap_client.set_options(timeout=300)

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    BatchID = []
    EventDate = []
    EventType = []
    SendID = []
    SubscriberKey = []
    TriggeredSendDefinitionObjectID = []

    print('>>> Retrieve Filtered SentEvents with GetMoreResults')
    getEvent = ET_Client.ET_SentEvent()
    getEvent.auth_stub = stubObj
    getEvent.props = ["BatchID","EventDate","EventType","SendID","SubscriberKey","TriggeredSendDefinitionObjectID"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'EventDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'EventDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      BatchID.append(Event['BatchID'])
      EventDate.append(Event['EventDate'])
      EventType.append(Event['EventType'])
      SendID.append(Event['SendID'])
      SubscriberKey.append(Event['SubscriberKey'])
      TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered SentEvents with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          BatchID.append(Event['BatchID'])
          EventDate.append(Event['EventDate'])
          EventType.append(Event['EventType'])
          SendID.append(Event['SendID'])
          SubscriberKey.append(Event['SubscriberKey'])
          TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered SentEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Fil

Turning lists into dataframe

In [ ]:
sent_df = pd.DataFrame({'BatchID':BatchID
                        ,'EventDate':EventDate
                        ,'EventType':EventType
                        ,'SendID':SendID
                        ,'SubscriberKey':SubscriberKey
                        ,'TriggeredSendID':TriggeredSendDefinitionObjectID})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(sent_df
                  , "laurent.merge_SentEvent"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 947.87it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.sent_event c
    USING (
      select *
      , concat(coalesce(cast(BatchID as string),"BatchNA"),"|",coalesce(cast(SendID as string),"SendNA"),"|",coalesce(TriggeredSendID,"TriggeredNA"),"|",SubscriberKey,"|",EventDate) as ID
      FROM laurent.merge_SentEvent) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.ID=tmp.ID
      , c.BatchID=tmp.BatchID
      , c.EventDate=tmp.EventDate
      , c.EventType=tmp.EventType
      , c.SendID=tmp.SendID
      , c.SubscriberKey=tmp.SubscriberKey
      , c.TriggeredSendID=tmp.TriggeredSendID
    WHEN NOT MATCHED THEN
      INSERT (ID,BatchID,EventDate,EventType,SendID,SubscriberKey,TriggeredSendID) VALUES (ID,BatchID,EventDate,EventType,SendID,SubscriberKey,TriggeredSendID)

Query is running:   0%|          |

""


# OpenEvent

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    ID = []
    BatchID = []
    CreatedDate = []
    EventDate = []
    EventType = []
    ModifiedDate = []
    SendID = []
    SubscriberKey = []
    TriggeredSendDefinitionObjectID = []

    print('>>> Retrieve Filtered OpenEvents with GetMoreResults')
    getEvent = ET_Client.ET_OpenEvent()
    getEvent.auth_stub = stubObj
    getEvent.props = ["ID","BatchID","CreatedDate","EventDate","EventType","ModifiedDate","SendID","SubscriberKey","TriggeredSendDefinitionObjectID"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'EventDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'EventDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      ID.append(Event['ID'])
      BatchID.append(Event['BatchID'])
      CreatedDate.append(Event['CreatedDate'])
      EventDate.append(Event['EventDate'])
      EventType.append(Event['EventType'])
      ModifiedDate.append(Event['ModifiedDate'])
      SendID.append(Event['SendID'])
      SubscriberKey.append(Event['SubscriberKey'])
      TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered OpenEvents with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          ID.append(Event['ID'])
          BatchID.append(Event['BatchID'])
          CreatedDate.append(Event['CreatedDate'])
          EventDate.append(Event['EventDate'])
          EventType.append(Event['EventType'])
          ModifiedDate.append(Event['ModifiedDate'])
          SendID.append(Event['SendID'])
          SubscriberKey.append(Event['SubscriberKey'])
          TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered OpenEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Fil

Turning lists into dataframe

In [ ]:
open_df = pd.DataFrame({'ID':ID
                        ,'BatchID':BatchID
                        ,'CreatedDate':CreatedDate
                        ,'EventDate':EventDate
                        ,'EventType':EventType
                        ,'ModifiedDate':ModifiedDate
                        ,'SendID':SendID
                        ,'SubscriberKey':SubscriberKey
                        ,'TriggeredSendID':TriggeredSendDefinitionObjectID})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(open_df
                  , "laurent.merge_OpenEvent"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 6413.31it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.open_event c
    USING (
      select * except (ID)
      , cast(ID as integer) as ID
      FROM laurent.merge_OpenEvent) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.ID=tmp.ID
      , c.BatchID=tmp.BatchID
      , c.CreatedDate=tmp.CreatedDate
      , c.EventDate=tmp.EventDate
      , c.EventType=tmp.EventType
      , c.ModifiedDate=tmp.ModifiedDate
      , c.SendID=tmp.SendID
      , c.SubscriberKey=tmp.SubscriberKey
      , c.TriggeredSendID=tmp.TriggeredSendID
    WHEN NOT MATCHED THEN
      INSERT (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID) VALUES (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID)

Query is running:   0%|          |

""


# ClickEvent

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    ID = []
    BatchID = []
    CreatedDate = []
    EventDate = []
    EventType = []
    ModifiedDate = []
    SendID = []
    SubscriberKey = []
    TriggeredSendDefinitionObjectID = []
    URL = []

    print('>>> Retrieve Filtered ClickEvents with GetMoreResults')
    getEvent = ET_Client.ET_ClickEvent()
    getEvent.auth_stub = stubObj
    getEvent.props = ["ID","BatchID","CreatedDate","EventDate","EventType","ModifiedDate","SendID","SubscriberKey","TriggeredSendDefinitionObjectID","URL"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'EventDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'EventDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      ID.append(Event['ID'])
      BatchID.append(Event['BatchID'])
      CreatedDate.append(Event['CreatedDate'])
      EventDate.append(Event['EventDate'])
      EventType.append(Event['EventType'])
      ModifiedDate.append(Event['ModifiedDate'])
      SendID.append(Event['SendID'])
      SubscriberKey.append(Event['SubscriberKey'])
      TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])
      URL.append(Event['URL'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered ClickEvents with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          ID.append(Event['ID'])
          BatchID.append(Event['BatchID'])
          CreatedDate.append(Event['CreatedDate'])
          EventDate.append(Event['EventDate'])
          EventType.append(Event['EventType'])
          ModifiedDate.append(Event['ModifiedDate'])
          SendID.append(Event['SendID'])
          SubscriberKey.append(Event['SubscriberKey'])
          TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])
          URL.append(Event['URL'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered ClickEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retr

Turning lists into dataframe

In [ ]:
click_df = pd.DataFrame({'ID':ID
                        ,'BatchID':BatchID
                        ,'CreatedDate':CreatedDate
                        ,'EventDate':EventDate
                        ,'EventType':EventType
                        ,'ModifiedDate':ModifiedDate
                        ,'SendID':SendID
                        ,'SubscriberKey':SubscriberKey
                        ,'TriggeredSendID':TriggeredSendDefinitionObjectID
                        ,'URL':URL})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(click_df
                  , "laurent.merge_ClickEvent"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 3368.92it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.click_event c
    USING (
      select * except (ID)
      , cast(ID as integer) as ID
      FROM laurent.merge_ClickEvent) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.BatchID=tmp.BatchID
      , c.CreatedDate=tmp.CreatedDate
      , c.EventDate=tmp.EventDate
      , c.EventType=tmp.EventType
      , c.ModifiedDate=tmp.ModifiedDate
      , c.SendID=tmp.SendID
      , c.SubscriberKey=tmp.SubscriberKey
      , c.TriggeredSendID=tmp.TriggeredSendID
      , c.URL=tmp.URL
    WHEN NOT MATCHED THEN
      INSERT (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID,URL) VALUES (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID,URL)

Query is running:   0%|          |

""


# BounceEvent

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    ID = []
    BatchID = []
    BounceCategory = []
    BounceType = []
    CreatedDate = []
    EventDate = []
    EventType = []
    ModifiedDate = []
    SendID = []
    SMTPCode = []
    SMTPReason = []
    SubscriberKey = []
    TriggeredSendDefinitionObjectID = []

    print('>>> Retrieve Filtered BounceEvents with GetMoreResults')
    getEvent = ET_Client.ET_BounceEvent()
    getEvent.auth_stub = stubObj
    getEvent.props = ["ID","BatchID","BounceCategory","BounceType","CreatedDate","EventDate","EventType","ModifiedDate","SendID","SMTPCode","SMTPReason","SubscriberKey","TriggeredSendDefinitionObjectID"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'EventDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'EventDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      ID.append(Event['ID'])
      BatchID.append(Event['BatchID'])
      BounceCategory.append(Event['BounceCategory'])
      BounceType.append(Event['BounceType'])
      CreatedDate.append(Event['CreatedDate'])
      EventDate.append(Event['EventDate'])
      EventType.append(Event['EventType'])
      ModifiedDate.append(Event['ModifiedDate'])
      SendID.append(Event['SendID'])
      SMTPCode.append(Event['SMTPCode'])
      SMTPReason.append(Event['SMTPReason'])
      SubscriberKey.append(Event['SubscriberKey'])
      TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered BounceEvents with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          ID.append(Event['ID'])
          BatchID.append(Event['BatchID'])
          BounceCategory.append(Event['BounceCategory'])
          BounceType.append(Event['BounceType'])
          CreatedDate.append(Event['CreatedDate'])
          EventDate.append(Event['EventDate'])
          EventType.append(Event['EventType'])
          ModifiedDate.append(Event['ModifiedDate'])
          SendID.append(Event['SendID'])
          SMTPCode.append(Event['SMTPCode'])
          SMTPReason.append(Event['SMTPReason'])
          SubscriberKey.append(Event['SubscriberKey'])
          TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered BounceEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered BounceEvents with GetMoreResults
Retrieve Status: True
MoreResults: False
Results Length: 1259


Turning lists into dataframe

In [ ]:
bounce_df = pd.DataFrame({'ID':ID
                        ,'BatchID':BatchID
                        ,'BounceCategory':BounceCategory
                        ,'BounceType':BounceType
                        ,'CreatedDate':CreatedDate
                        ,'EventDate':EventDate
                        ,'EventType':EventType
                        ,'ModifiedDate':ModifiedDate
                        ,'SendID':SendID
                        ,'SMTPCode':SMTPCode
                        ,'SMTPReason':SMTPReason
                        ,'SubscriberKey':SubscriberKey
                        ,'TriggeredSendID':TriggeredSendDefinitionObjectID})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(bounce_df
                  , "laurent.merge_BounceEvent"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 8081.51it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.bounce_event c
    USING (
      select * except (SMTPCode)
      , cast(SMTPCode as integer) as SMTPCode
      FROM laurent.merge_BounceEvent) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.BatchID=tmp.BatchID
      , c.BounceCategory=tmp.BounceCategory
      , c.BounceType=tmp.BounceType
      , c.CreatedDate=tmp.CreatedDate
      , c.EventDate=tmp.EventDate
      , c.EventType=tmp.EventType
      , c.ModifiedDate=tmp.ModifiedDate
      , c.SendID=tmp.SendID
      , c.SMTPCode=tmp.SMTPCode
      , c.SMTPReason=tmp.SMTPReason
      , c.SubscriberKey=tmp.SubscriberKey
      , c.TriggeredSendID=tmp.TriggeredSendID
    WHEN NOT MATCHED THEN
      INSERT (ID,BatchID,BounceCategory,BounceType,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SMTPCode,SMTPReason,SubscriberKey,TriggeredSendID) VALUES (ID,BatchID,BounceCategory,BounceType,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SMTPCode,SMTPReason,SubscriberKey,TriggeredSendID)

Query is running:   0%|          |

""


# UnsubEvent

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    ID = []
    BatchID = []
    CreatedDate = []
    EventDate = []
    EventType = []
    ModifiedDate = []
    SendID = []
    SubscriberKey = []
    TriggeredSendDefinitionObjectID = []

    print('>>> Retrieve Filtered UnsubEvents with GetMoreResults')
    getEvent = ET_Client.ET_UnsubEvent()
    getEvent.auth_stub = stubObj
    getEvent.props = ["ID","BatchID","CreatedDate","EventDate","EventType","ModifiedDate","SendID","SubscriberKey","TriggeredSendDefinitionObjectID"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'EventDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'EventDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      ID.append(Event['ID'])
      BatchID.append(Event['BatchID'])
      CreatedDate.append(Event['CreatedDate'])
      EventDate.append(Event['EventDate'])
      EventType.append(Event['EventType'])
      ModifiedDate.append(Event['ModifiedDate'])
      SendID.append(Event['SendID'])
      SubscriberKey.append(Event['SubscriberKey'])
      TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered UnsubEvents with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          ID.append(Event['ID'])
          BatchID.append(Event['BatchID'])
          CreatedDate.append(Event['CreatedDate'])
          EventDate.append(Event['EventDate'])
          EventType.append(Event['EventType'])
          ModifiedDate.append(Event['ModifiedDate'])
          SendID.append(Event['SendID'])
          SubscriberKey.append(Event['SubscriberKey'])
          TriggeredSendDefinitionObjectID.append(Event['TriggeredSendDefinitionObjectID'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered UnsubEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered UnsubEvents with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered UnsubEvents with GetMoreResults
Retrieve Status: True
MoreResults: False
Results Length: 1232


Turning lists into dataframe

In [ ]:
unsub_df = pd.DataFrame({'ID':ID
                        ,'BatchID':BatchID
                        ,'CreatedDate':CreatedDate
                        ,'EventDate':EventDate
                        ,'EventType':EventType
                        ,'ModifiedDate':ModifiedDate
                        ,'SendID':SendID
                        ,'SubscriberKey':SubscriberKey
                        ,'TriggeredSendID':TriggeredSendDefinitionObjectID})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(unsub_df
                  , "laurent.merge_UnsubEvent"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 6105.25it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.unsub_event c
    USING (
      select *
      FROM laurent.merge_UnsubEvent) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.BatchID=tmp.BatchID
      , c.CreatedDate=tmp.CreatedDate
      , c.EventDate=tmp.EventDate
      , c.EventType=tmp.EventType
      , c.ModifiedDate=tmp.ModifiedDate
      , c.SendID=tmp.SendID
      , c.SubscriberKey=tmp.SubscriberKey
      , c.TriggeredSendID=tmp.TriggeredSendID
    WHEN NOT MATCHED THEN
      INSERT (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID) VALUES (ID,BatchID,CreatedDate,EventDate,EventType,ModifiedDate,SendID,SubscriberKey,TriggeredSendID)

Query is running:   0%|          |

""


# Send

In [ ]:
try:
    stubObj = ET_Client.ET_Client(
  False, False,
  {
      'clientid': 'vswwvn1xc8puqc7xhacw23eo',
      'clientsecret': 'Axa3oXGfiKQpEnI6Vd5l1Zl5',
      'authenticationurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.auth.marketingcloudapis.com/',
      'baseapiurl': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.rest.marketingcloudapis.com/',
      'soapendpoint': 'https://mc9hhw52jkm86f1dwfdsjmfljl24.soap.marketingcloudapis.com/',
      'useOAuth2Authentication': 'True',
      'applicationType': 'server'
  })

    ## Modify the date below to reduce the number of results returned from the request
    ## Setting this too far in the past could result in a very large response size
    startDate = past_7_days_formatted
    endDate = today_formatted

    ID = []
    BCCEmail = []
    CreatedDate = []
    ModifiedDate = []
    EmailId = []
    SendDate = []
    FromAddress = []
    FromName = []
    Duplicates = []
    InvalidAddresses = []
    ExistingUndeliverables = []
    ExistingUnsubscribes = []
    HardBounces = []
    SoftBounces = []
    OtherBounces = []
    ForwardedEmails = []
    UniqueClicks = []
    UniqueOpens = []
    NumberSent = []
    NumberDelivered = []
    Unsubscribes = []
    MissingAddresses = []
    Subject = []
    PreviewURL = []
    SentDate = []
    EmailName = []
    Status = []
    IsMultipart = []
    IsAlwaysOn = []
    NumberTargeted = []
    NumberErrored = []
    NumberExcluded = []

    print('>>> Retrieve Filtered Send with GetMoreResults')
    getEvent = ET_Client.ET_Send()
    getEvent.auth_stub = stubObj
    # getEvent.props = ["ID","BccEmail","CreatedDate","ModifiedDate","Email","SendDate","FromAddress","FromName","Duplicates","InvalidAddresses","ExistingUndeliverables","ExistingUnsubscribes","HardBounces","SoftBounces","OtherBounces","ForwardedEmails","UniqueClicks","UniqueOpens","NumberSent","NumberDelivered","Unsubscribes","MissingAddresses","Subject","PreviewURL","SentDate","EmailName","Status","IsMultipart","IsAlwaysOn","NumberTargeted","NumberErrored","NumberExcluded"]
    getEvent.search_filter =  {'LeftOperand' : {'Property' : 'ModifiedDate','SimpleOperator' : 'greaterThanOrEqual','DateValue' : startDate}
                                   , 'RightOperand' : {'Property' : 'ModifiedDate','SimpleOperator' : 'lessThan','DateValue' : endDate}
                                   , 'LogicalOperator' : 'AND'}
    getResponse = getEvent.get()
    print('Retrieve Status: ' + str(getResponse.status))
    # print('Code: ' + str(getResponse.code))
    # print('Message: ' + str(getResponse.message))
    print('MoreResults: ' + str(getResponse.more_results))
    # print('RequestID: ' + str(getResponse.request_id))
    print('Results Length: ' + str(len(getResponse.results)))

    ResponseResults = getResponse.results

    for Event in ResponseResults:
      ID.append(Event['ID'])
      BCCEmail.append(None)
      CreatedDate.append(Event['CreatedDate'])
      ModifiedDate.append(Event['ModifiedDate'])
      EmailId.append(Event['Email']['ID'])
      SendDate.append(Event['SendDate'])
      FromAddress.append(Event['FromAddress'])
      FromName.append(Event['FromName'])
      Duplicates.append(Event['Duplicates'])
      InvalidAddresses.append(Event['InvalidAddresses'])
      ExistingUndeliverables.append(Event['ExistingUndeliverables'])
      ExistingUnsubscribes.append(Event['ExistingUnsubscribes'])
      HardBounces.append(Event['HardBounces'])
      SoftBounces.append(Event['SoftBounces'])
      OtherBounces.append(Event['OtherBounces'])
      ForwardedEmails.append(Event['ForwardedEmails'])
      UniqueClicks.append(Event['UniqueClicks'])
      UniqueOpens.append(Event['UniqueOpens'])
      NumberSent.append(Event['NumberSent'])
      NumberDelivered.append(Event['NumberDelivered'])
      Unsubscribes.append(Event['Unsubscribes'])
      MissingAddresses.append(Event['MissingAddresses'])
      Subject.append(Event['Subject'])
      PreviewURL.append(Event['PreviewURL'])
      SentDate.append(None)
      EmailName.append(Event['EmailName'])
      Status.append(Event['Status'])
      IsMultipart.append(Event['IsMultipart'])
      IsAlwaysOn.append(Event['IsAlwaysOn'])
      NumberTargeted.append(Event['NumberTargeted'])
      NumberErrored.append(Event['NumberErrored'])
      NumberExcluded.append(Event['NumberExcluded'])

    while getResponse.more_results:
        print('>>> Continue Retrieve Filtered Send with GetMoreResults')
        getResponse = getEvent.getMoreResults()
        print('Retrieve Status: ' + str(getResponse.status))
        # print('Code: ' + str(getResponse.code))
        # print('Message: ' + str(getResponse.message))
        print('MoreResults: ' + str(getResponse.more_results))
        # print('RequestID: ' + str(getResponse.request_id))
        print('Results Length: ' + str(len(getResponse.results)))

        ResponseResults = getResponse.results

        for Event in ResponseResults:
          ID.append(Event['ID'])
          BCCEmail.append(None)
          CreatedDate.append(Event['CreatedDate'])
          ModifiedDate.append(Event['ModifiedDate'])
          EmailId.append(Event['Email']['ID'])
          SendDate.append(Event['SendDate'])
          FromAddress.append(Event['FromAddress'])
          FromName.append(Event['FromName'])
          Duplicates.append(Event['Duplicates'])
          InvalidAddresses.append(Event['InvalidAddresses'])
          ExistingUndeliverables.append(Event['ExistingUndeliverables'])
          ExistingUnsubscribes.append(Event['ExistingUnsubscribes'])
          HardBounces.append(Event['HardBounces'])
          SoftBounces.append(Event['SoftBounces'])
          OtherBounces.append(Event['OtherBounces'])
          ForwardedEmails.append(Event['ForwardedEmails'])
          UniqueClicks.append(Event['UniqueClicks'])
          UniqueOpens.append(Event['UniqueOpens'])
          NumberSent.append(Event['NumberSent'])
          NumberDelivered.append(Event['NumberDelivered'])
          Unsubscribes.append(Event['Unsubscribes'])
          MissingAddresses.append(Event['MissingAddresses'])
          Subject.append(Event['Subject'])
          PreviewURL.append(Event['PreviewURL'])
          SentDate.append(None)
          EmailName.append(Event['EmailName'])
          Status.append(Event['Status'])
          IsMultipart.append(Event['IsMultipart'])
          IsAlwaysOn.append(Event['IsAlwaysOn'])
          NumberTargeted.append(Event['NumberTargeted'])
          NumberErrored.append(Event['NumberErrored'])
          NumberExcluded.append(Event['NumberExcluded'])

except Exception as e:
    print('Caught exception: ' + e.message)
    print(e)

>>> Retrieve Filtered Send with GetMoreResults
Retrieve Status: True
MoreResults: True
Results Length: 2500
>>> Continue Retrieve Filtered Send with GetMoreResults
Retrieve Status: True
MoreResults: False
Results Length: 58


Turning lists into dataframe

In [ ]:
send_df = pd.DataFrame({"ID":ID
                        ,"BCCEmail":BCCEmail
                        ,"CreatedDate":CreatedDate
                        ,"ModifiedDate":ModifiedDate
                        ,"EmailId":EmailId
                        ,"SendDate":SendDate
                        ,"FromAddress":FromAddress
                        ,"FromName":FromName
                        ,"Duplicates":Duplicates
                        ,"InvalidAddresses":InvalidAddresses
                        ,"ExistingUndeliverables":ExistingUndeliverables
                        ,"ExistingUnsubscribes":ExistingUnsubscribes
                        ,"HardBounces":HardBounces
                        ,"SoftBounces":SoftBounces
                        ,"OtherBounces":OtherBounces
                        ,"ForwardedEmails":ForwardedEmails
                        ,"UniqueClicks":UniqueClicks
                        ,"UniqueOpens":UniqueOpens
                        ,"NumberSent":NumberSent
                        ,"NumberDelivered":NumberDelivered
                        ,"Unsubscribes":Unsubscribes
                        ,"MissingAddresses":MissingAddresses
                        ,"Subject":Subject
                        ,"PreviewURL":PreviewURL
                        ,"SentDate":SentDate
                        ,"EmailName":EmailName
                        ,"Status":Status
                        ,"IsMultipart":IsMultipart
                        ,"IsAlwaysOn":IsAlwaysOn
                        ,"NumberTargeted":NumberTargeted
                        ,"NumberErrored":NumberErrored
                        ,"NumberExcluded":NumberExcluded})

Then create related table to a temp dataset

In [ ]:
pandas_gbq.to_gbq(send_df
                  , "laurent.merge_send"
                  , project_id="upd-ecommerce-data"
                  , if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 3675.99it/s]


In [ ]:
%%bigquery --project upd-ecommerce-data
MERGE salesforce_marketing_cloud.send c
    USING (
      select * except (SentDate)
      , cast(SentDate as timestamp) as SentDate
      FROM laurent.merge_send) tmp
    ON c.ID = tmp.ID
    WHEN MATCHED THEN
      UPDATE SET c.BCCEmail=tmp.BCCEmail
      ,c.CreatedDate=tmp.CreatedDate
      ,c.ModifiedDate=tmp.ModifiedDate
      ,c.EmailId=tmp.EmailId
      ,c.SendDate=tmp.SendDate
      ,c.FromAddress=tmp.FromAddress
      ,c.FromName=tmp.FromName
      ,c.Duplicates=tmp.Duplicates
      ,c.InvalidAddresses=tmp.InvalidAddresses
      ,c.ExistingUndeliverables=tmp.ExistingUndeliverables
      ,c.ExistingUnsubscribes=tmp.ExistingUnsubscribes
      ,c.HardBounces=tmp.HardBounces
      ,c.SoftBounces=tmp.SoftBounces
      ,c.OtherBounces=tmp.OtherBounces
      ,c.ForwardedEmails=tmp.ForwardedEmails
      ,c.UniqueClicks=tmp.UniqueClicks
      ,c.UniqueOpens=tmp.UniqueOpens
      ,c.NumberSent=tmp.NumberSent
      ,c.NumberDelivered=tmp.NumberDelivered
      ,c.Unsubscribes=tmp.Unsubscribes
      ,c.MissingAddresses=tmp.MissingAddresses
      ,c.Subject=tmp.Subject
      ,c.PreviewURL=tmp.PreviewURL
      ,c.SentDate=tmp.SentDate
      ,c.EmailName=tmp.EmailName
      ,c.Status=tmp.Status
      ,c.IsMultipart=tmp.IsMultipart
      ,c.IsAlwaysOn=tmp.IsAlwaysOn
      ,c.NumberTargeted=tmp.NumberTargeted
      ,c.NumberErrored=tmp.NumberErrored
      ,c.NumberExcluded=tmp.NumberExcluded
    WHEN NOT MATCHED THEN
      INSERT (ID,BCCEmail,CreatedDate,ModifiedDate,EmailId,SendDate,FromAddress,FromName,Duplicates,InvalidAddresses,ExistingUndeliverables,ExistingUnsubscribes,HardBounces,SoftBounces,OtherBounces,ForwardedEmails,UniqueClicks,UniqueOpens,NumberSent,NumberDelivered,Unsubscribes,MissingAddresses,Subject,PreviewURL,SentDate,EmailName,Status,IsMultipart,IsAlwaysOn,NumberTargeted,NumberErrored,NumberExcluded) VALUES (ID,BCCEmail,CreatedDate,ModifiedDate,EmailId,SendDate,FromAddress,FromName,Duplicates,InvalidAddresses,ExistingUndeliverables,ExistingUnsubscribes,HardBounces,SoftBounces,OtherBounces,ForwardedEmails,UniqueClicks,UniqueOpens,NumberSent,NumberDelivered,Unsubscribes,MissingAddresses,Subject,PreviewURL,SentDate,EmailName,Status,IsMultipart,IsAlwaysOn,NumberTargeted,NumberErrored,NumberExcluded)

Query is running:   0%|          |

""
